In [47]:
import pandas as pd
from datetime import datetime
from slugify import slugify

In [48]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

In [49]:
now = datetime.now()
dt_string = now.strftime("%Y-%m-%d %H:%M:%S")

In [50]:
df = pd.read_csv('csv/products.csv', delimiter=';')
priceDf = pd.read_csv('out/price.csv')
usersDf = pd.read_csv('csv/users.csv')
users = pd.read_csv('out/users.csv')
locDf = pd.read_csv('out/localities.csv', delimiter=';')
prodDf = pd.read_csv('out/products.csv')

In [51]:
df1 = df[['Price 1', 'Curr1', 'Number Review 1', 'Review 1', 'Rating 1', 'ID', 'Name']]
df2 = df[['Price 2', 'Curr2', 'Number Review 2', 'Review 2', 'Rating 2', 'ID', 'Name']]
df3 = df[['Price 3', 'Curr3', 'Number Review 3', 'Review 3', 'Rating 3', 'ID', 'Name']]
df4 = df[['Price 4', 'Curr4', 'Number Review 4', 'Review 4', 'Rating 4', 'ID', 'Name']]
df5 = df[['Price 5', 'Curr5', 'Number Review 5', 'Review 5', 'Rating 5', 'ID', 'Name']]

In [52]:
df1 = df1.rename(columns={'Price 1' : 'Price', 'Curr1' : 'Curr', 'Number Review 1' : 'Number Review', \
                          'Review 1' : 'Review', 'Rating 1' : 'Rating'})
df2 = df2.rename(columns={'Price 2' : 'Price', 'Curr2' : 'Curr', 'Number Review 2' : 'Number Review', \
                          'Review 2' : 'Review', 'Rating 2' : 'Rating'})
df3 = df3.rename(columns={'Price 3' : 'Price', 'Curr3' : 'Curr', 'Number Review 3' : 'Number Review', \
                          'Review 3' : 'Review', 'Rating 3' : 'Rating'})
df4 = df4.rename(columns={'Price 4' : 'Price', 'Curr4' : 'Curr', 'Number Review 4' : 'Number Review', \
                          'Review 4' : 'Review', 'Rating 4' : 'Rating'})
df5 = df5.rename(columns={'Price 5' : 'Price', 'Curr5' : 'Curr', 'Number Review 5' : 'Number Review', \
                          'Review 5' : 'Review', 'Rating 5' : 'Rating'})

In [53]:
newDf = pd.concat([df1, df2, df3, df4, df5])

In [54]:
newDf.rename(columns = {'Review':'review', 'Rating' : 'rating'}, inplace = True)

In [55]:
columns = ['id', 'createdDate', 'updatedDate', 'name', 'internalNote', 'isActive', 'authorId', 'statusId', \
          'productId', 'priceId', 'placeId', 'slug']

In [56]:
newDf = pd.concat([newDf, pd.DataFrame(columns = columns)])

In [57]:
import random
import time
 
 
def str_time_prop(start, end, time_format, prop):
    stime = time.mktime(time.strptime(start, time_format))
    etime = time.mktime(time.strptime(end, time_format))
    ptime = stime + prop * (etime - stime)
    return time.strftime(time_format, time.localtime(ptime))
 
 
def random_date(start, end, prop):
    return str_time_prop(start, end, '%Y-%m-%d', prop)

In [58]:
newDf['createdDate'] = [random_date("2021-1-1", "2022-4-30", random.random()) for i in range(1, newDf.shape[0] + 1)]
newDf['updatedDate'] = [i for i in newDf['createdDate']]
newDf['name'] = ''
newDf['internalNote'] = ''
newDf['isActive'] = True
newDf['statusId'] = 4
newDf['placeId'] = ''

In [59]:
def set_slug(value):
    return slugify(str(value))

newDf['slug'] = newDf['review'][:10].apply(set_slug)

In [60]:
priceDf['merged'] = priceDf['amount'] + priceDf['currencyId']
newDf['merged'] = newDf['Price'] + newDf['Curr']

In [61]:
def get_price_id(value):
    try:
        return priceDf[:][priceDf['merged'] == str(value)]['id'].values[0]
    except:
        return ''

newDf['priceId'] = newDf['merged'].apply(get_price_id)
#newDf['priceId'] = ''

In [62]:
def get_product_id(value):
    try:
        return prodDf[:][prodDf['name'] == str(value)]['id'].values[0]
    except:
        return ''

newDf['productId'] = newDf['Name'].apply(get_product_id)

In [63]:
def get_author_id(value):
    try:
        temp = usersDf[:][(usersDf['Review1'] == str(value)) | \
         (usersDf['Review2'] == str(value)) | \
         (usersDf['Review3'] == str(value)) | \
         (usersDf['Review4'] == str(value)) | \
         (usersDf['Review5'] == str(value))]['Username'].values[0]
        return users[:][users['username'] == temp]['id'].values[0]
    except:
        return ''

newDf['authorId'] = newDf['Number Review'].apply(get_author_id)
#newDf['authorId'] = 13

In [64]:
newDf = newDf.drop(['Price', 'Curr', 'Number Review', 'ID', 'merged', 'Name'], axis=1)

In [65]:
newDf['rating'] = newDf['rating'].fillna(0)
newDf['id'] = [i for i in range(1, newDf.shape[0]+1)]

In [66]:
newDf.shape

(9715, 14)

In [67]:
newDf.to_csv('out/stories.csv', index=False, encoding='utf-8')

In [68]:
newDf.tail()

,review,rating,id,createdDate,updatedDate,name,internalNote,isActive,authorId,statusId,productId,priceId,placeId,slug
1938,The taxi ride from Dubai Airport to Dubai city...,0.0,9711,2021-04-03,2021-04-03,,,True,1818,4,1939,1897,,NaN
1939,The bus ticket costs Dhs 7 and you can pay in ...,0.0,9712,2021-03-02,2021-03-02,,,True,1607,4,1940,1902,,NaN
1940,NaN,0.0,9713,2022-04-29,2022-04-29,,,True,,4,1941,,,NaN
1941,NaN,0.0,9714,2022-02-20,2022-02-20,,,True,,4,1942,,,NaN
1942,The fastest way to travel from Antananarivo to...,0.0,9715,2021-06-29,2021-06-29,,,True,2324,4,1943,1793,,NaN
